In [ ]:
!wget https://github.com/d-uni/LPPL_model/archive/refs/heads/main.zip
!unzip main.zip
%cd LPPL_model-main

In [ ]:
# !git clone https://github.com/d-uni/LPPL_model.git
# %cd LPPL_model

In [ ]:
from model_lppls import ModelLPPLS
from rolling_calibrator import RollingLPPLCalibrator
from distribution_calibrators import (
    DistributionLPPLCalibrator_for_different_Windows,
    DistributionLPPLCalibrator_for_different_Dates,
)
from visualization import plot_lppls_results, plot_tc_distribution,plot_combined_tc_distributions

# Test model_lppls.py

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

data = yf.download("000001.SS", start="2007-01-01", end="2008-01-01")
price_series = data["Close"].dropna()

t1 = "2007-03-12"
t2 = "2007-10-10"
fit_window = price_series[t1:t2]


dates = pd.to_datetime(fit_window.index)
t = (dates - dates[0]).days / 365.25  # time in years since start
p = fit_window.values  # price values

model = ModelLPPLS(t, p)
model.set_calibration_date(t2)

model.fit_multistart()
model.summary()

[*********************100%***********************]  1 of 1 completed


NameError: name 'ModelLPPLS' is not defined

# Test rolling_calibrator.py

In [ ]:
# --- Download data ---
SYMBOL = "^GSPC"
START_DATE, END_DATE = "2020-01-01", "2025-10-29"
data_rolling_calibrator  = yf.download(SYMBOL, start=START_DATE, end=END_DATE, progress=False, auto_adjust=True)["Close"].dropna()

data = data_rolling_calibrator
t_series = (data.index - data.index[0]).days / 365.25
p_series = data.values.squeeze()



rolling_calibrator = RollingLPPLCalibrator(t_series, p_series)

# ------ Acceptance thresholds
rolling_calibrator.set_acceptance_thresholds(
    r2_min=0.95, rmse_max=0.04, kappa_min=0.05, tc_horizon_years=0.5
)

result_rolling_calibrator = rolling_calibrator.run(
    start_roll = 1.0,       # start of rolling period (years)
    end_roll = 4.0,         # end of rolling period (years)
    window_years = 1.0,     # window size
    step_years = 4 / 365.25 # step = 2 days
)

In [ ]:
result_rolling_calibrator.head()

# Test distribution_calibrators.py

In [ ]:
target_date_ = pd.Timestamp("2010-01-01")

SYMBOL = "^GSPC"
START_DATE, END_DATE = "2006-01-01", "2011-01-01"
data_tc_distribution_over_window = yf.download(SYMBOL, start=START_DATE, end=END_DATE, progress=False, auto_adjust=True)["Close"].dropna()

data = data_tc_distribution_over_window
t_series = (data.index - data.index[0]).days / 365.25
p_series = data.values.squeeze()

calibrator = DistributionLPPLCalibrator_for_different_Windows(t_series, p_series)

# --- Acceptance thresholds ---
calibrator.set_acceptance_thresholds(
    r2_min=0.9,
    rmse_max=0.06,
    kappa_min=0.05,
    tc_horizon_years=0.5
)

target_date = (target_date_ - data.index[0]).days / 365.25 #!!!!!!! <- <- <- <- <- <- CONSIDER TARGET DAY

result_tc_distribution_over_window = calibrator.tc_distribution_over_window(
    target_date=target_date,
    min_window_years=1/12,  # 1 month
    max_window_years=1.0,   # 1 year
    step_window_years=1/365.25, # increment by 1 day
    acceptance_thresholds=True # IF you put False -> you don't take into account the thresholds, presented earlier
)

In [ ]:
result_tc_distribution_over_window.head()

In [ ]:
# --- Download data ---
SYMBOL = "^GSPC"
START_DATE, END_DATE = "2006-01-01", "2011-01-01"
data_tc_distribution_over_dates = yf.download(SYMBOL, start=START_DATE, end=END_DATE, progress=False, auto_adjust=True)["Close"].dropna()

data = data_tc_distribution_over_dates
t_series = (data.index - data.index[0]).days / 365.25
p_series = data.values.squeeze()

# --- Initialize calibrator ---
calibrator = DistributionLPPLCalibrator_for_different_Dates(t_series, p_series)
calibrator.set_acceptance_thresholds(r2_min=0.9, rmse_max=0.06, kappa_min=0.05, tc_horizon_years=0.5)

# --- Parameters ---
target_date = (target_date_ - data.index[0]).days / 365.25
window_years = 1.0                 # fixed calibration window
distribution_window_years = 1/12    # look 6 months before target date
step_years = 1 / 365.25            # 5 days between calibrations

# --- Run distribution over calibration dates ---
result_tc_distribution_over_dates = calibrator.tc_distribution_over_dates(
    target_date=target_date,
    window_years=window_years,
    distribution_window_years=distribution_window_years,
    step_years=step_years,
    acceptance_thresholds=True
)

In [ ]:
result_tc_distribution_over_dates.head()

# Test visualization.py

In [ ]:
plot_lppls_results(data_rolling_calibrator, result_rolling_calibrator)

In [ ]:
plot_lppls_results(data_tc_distribution_over_window, result_tc_distribution_over_window)

In [ ]:
plot_lppls_results(data_tc_distribution_over_dates, result_tc_distribution_over_dates)

In [ ]:
tc_predicted_from_start_dates = result_tc_distribution_over_dates["tc"]-(target_date_ - data_tc_distribution_over_dates.index[0]).days / 365.25

In [ ]:
plot_tc_distribution(tc_predicted_from_start_dates, symbol=SYMBOL)

In [ ]:
tc_predicted_from_start_window = result_tc_distribution_over_window["tc"]-(target_date_ - data_tc_distribution_over_window.index[0]).days / 365.25

In [ ]:
result_tc_distribution_over_window["tc"]

In [ ]:
data_tc_distribution_over_window.index[0]

In [ ]:
plot_tc_distribution(tc_predicted_from_start_window, symbol=SYMBOL)

In [ ]:
plot_combined_tc_distributions(tc_predicted_from_start_dates, tc_predicted_from_start_window, symbol=SYMBOL)